In [25]:
import pandas as pd
import re
import nltk

# to clean data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter,OrderedDict
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist
nltk.download('all')

nltk.download('stopwords')
stop = stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anape\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anape\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anape\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\anape\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\anape\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\anape\AppData\Roaming\nltk_data...
[nltk_dat

In [124]:
df =  pd.read_csv('tweet_all2.csv',  sep=';;', on_bad_lines='skip',lineterminator='\r', engine ='python')
df_jan =  pd.read_csv('JanTweet.csv',  sep=';;', on_bad_lines='skip',lineterminator='\r', engine ='python')

# Rename columns in df_jan
new_column_names = {'"id"': 'id', '"text"': 'text', '"created_at"': 'created_at', '"author_id"':'author_id', '"lang"': 'lang', '"in_reply_to_user_id"':'in_reply_to_user_id'}  # Provide a mapping of old column names to new column names
df_jan = df_jan.rename(columns=new_column_names)

# Concatenate the two DataFrames vertically
concatenated_df = pd.concat([df_jan, df])

# Print the length of the concatenated DataFrame
#print("Length of the concatenated DataFrame:", len(concatenated_df))
print(concatenated_df.tail(5))

                         id  \
1170847  999981781209280512   
1170848  999981813505429505   
1170849  999981819981377536   
1170850  999982136965902338   
1170851  999982870675578880   

                                                      text  \
1170847  BH ta um caos sos quero voltar p meu país de i...   
1170848  RT @Re_Carvalho89: Rodoanel com protesto de ca...   
1170849  RT @Re_Carvalho89: Rodoanel com protesto de ca...   
1170850  Gasolina hunters de SP: tem 3 postos na Bandei...   
1170851  @_ecovias Bom dia gostaria de saber se a imigr...   

                  created_at   author_id lang in_reply_to_user_id  
1170847  2018-05-25 11:53:30   277210631   pt                None  
1170848  2018-05-25 11:53:37   144407784   pt                None  
1170849  2018-05-25 11:53:39   142847293   pt                None  
1170850  2018-05-25 11:54:54  2568000512   pt                None  
1170851  2018-05-25 11:57:49   201444466   pt           138075168  


# 1 - Cleaning and Text Processing

MAINTAIN THE ROWS WITH SELECTED #

In [125]:
selected_hashtags = ['#discriminação', '#corridoiumanitari', '#accoglienza', '#refugiadosvavel', '#freeopenarms', '#iwelcome',
    '#integração', '#atletasrefugiados', '#refugioderechoshumanos', '#volemacollir', '#nobordersnonations', '#visto', 
    '#opentheborders', '#eupertenço', '#sonbienvenidos', '#refuxiadas', '#leidemigraçaonao', '#comosrefugiados', 
    '#cambioclimático', '#refugeeday', '#prayforsyria', '#worldrefugeeday2018', '#formigration', '#crisehumanitária', 
    '#vetatemermigração', '#overthefortress', '#xenofobianão', '#orepelosrefugiados', '#fechaasfronteirasportugal', 
    '#diainternacionaldosmigrantes', '#migracaonao', '#crisederefugiados', '#meuamigorefugiadocopa', '#haitianosficamnobrasil', 
    '#xenófobas', '#crisis', '#syriacrisis', '#teamrefugees', '#aloysiomigracaoagoranao', '#imigracaononordeste', '#deportação',
    '#vetatemer', '#refugeeswelcomegfbpa', '#safepassage', '#exodus', '#eusourefugio', '#opacolhida',
    '#refugee', '#jantarhumanitario', '#acogida', '#minutorefugiados', '#sirios', '#maismédicos', '#sírios', '#expressentry', 
    '#stopracismo', '#haitianos', '#somostodosimigrantes', '#openeuborders', '#moçambicanos', '#acnur', '#terroristaleimigracao', 
    '#standwithiraqirefugees', '#sharehumanity', '#derechoshumanos', '#welcomerefugees', '#frontex', '#fronteira', 
    '#diversidadcultural', '#artemigrante', '#racismo', '#acolhidas', '#venezuelanas', '#cáritas', '#diamundialhumanitário',
    '#diamundialhumanitario', '#caritasrj', '#venezuelanos', '#naoaleidemigracao', '#direitos', '#migramundo', 
    '#imigraçãoitaliana', '#elrefugiado', '#refuxiados', '#operaçãoacolhida', '#crisisrefugiados', '#naoaceitamosrefugiados',
    '#somosmigrantes', '#operacaoacolhida', '#prouracisme', '#amorsemfronteiras', '#caravanamigrante', '#refugeeolympicteam', 
    '#festadoimigrante', '#humanflow', '#cáritasbrasileira', '#conlosrefugiados', '#muslimban', '#assistência', '#withsyria', 
    '#pactoglobalsobrerefugiados', '#diamundialdorefugiado', '#onu', '#deportaci', '#worldrefugeeday', '#criminiimmigrati',
    '#reassentados', '#semananacionaldosrefugiados', '#vistos', '#refugeeforum', '#prayforsyria', '#venezuelanos', '#yorefugio',
    '#teamrefugee', '#somostodosimigrantes', '#openborders', '#venezuela', '#pactoglobalsobrerefugiados', '#respeitaosrefugiados', 
    '#formigration', '#aloysiomigraçãoagoranao', '#eupertenço', '#rejeiteleidemigracao', '#bienvenidos', '#refugioderechoshumanos', 
    '#crisis', '#imigracao', '#naoaceitamosrefugiados', '#vetatemer', '#seawatch', '#muros', '#caritas', '#terroristaleimigracao', 
    '#ibelong', '#imigracaoitaliana', '#nãoaceitamosrefugiados', '#refugiada', '#estrangeiros', '#refugiadas', '#imigrantes',
    '#leidamigraçãonão', '#refugiadoscristaos', '#stopracismo', '#operaçãoacolhida', '#operativomigrante', '#prayforaleppo', 
    '#xenophobia', '#crisederefugiados', '#migrantsday', '#crisenavenezuela', '#refugiados', '#eumigrante', '#acolhidas', 
    '#refúgioemnúmeros', '#migración', '#caravanamigrante', '#amorsemfronteiras', '#jeansforrefugees', '#compartilhehumanidade', 
    '#xenófobas', '#acogida', '#ueturkey', '#refuxiados', '#imigracaononordeste', '#syriacrisis', '#rejeitemigracao', 
    '#teamrefugees', '#sírios', '#onu', '#orepelosrefugiados', '#semfronteiras', '#exodus', '#refugeeolympicteam', 
    '#diainternacionaldosmigrantes', '#naosetrataapenasdemigrantes', '#imigraçãogerainovação', '#direitoshumanos', 
    '#venezuelanas', '#unhcr', '#refugeats', '#fronteira', '#humanrights', '#vetatemerleiimigracao', '#criminiimmigrati',
    '#welcomerefugees', '#todossomosrefugiados', '#xenofobia', '#imigraçãoitaliana', '#estamosxlosrefugiados', 
    '#migration', '#imigrante', '#accoglienza', '#imigração', '#crisehumanitária', '#worldrefugeeday', '#withsyria', 
    '#refugiado', '#refugiadosnobrasil', '#immigration', '#visto', '#aldeiadosimigrantes', 
    '#pactoglobalimigracao', '#migracaonao', '#jantarhumanitario', '#refugeeswelcome', '#migração', '#diamundialhumanitário', 
    '#refugeeswelcomegfbpa', '#cáritasbrasileira', '#refúgio', '#crisisderefugiados', '#safepassage', '#migraciones', 
    '#vetaimigracaotemer', '#syria', '#cáritas', '#elrefugiado', '#refugees', '#teamrefugess', '#sirios', '#muslimban', 
    '#migrações', '#rifugiato', '#venezolanos', '#migracoes', '#syrianrefugees', '#volunteer', '#gratidão', 
    '#proudlyrefugee', '#refugeerights', '#withrefugees', '#migrants', '#refugeelivesmatter', '#nooneisillegal', 
    '#nonprofit', '#forrefugees', '#migrationmatters', '#refugeestories', '#humanitarian', 
    '#standwithrefugees', '#humanity', '#inclusion', '#everydayrefugees', '#refugeecrisis', '#refugeechildren', 
    '#asylumseekers', '#helprefugees', '#worldrefugeeday', '#refugeesupport', '#refugeeawareness', '#refugeeaction', 
    '#childrenofrefugees', '#refugeeyouth', '#fugeeswelcome', '#refugeetent', '#refugeestrong', '#educationforall', 
    '#refugeeswelcome', '#refugeeresettlement', '#forcedmigration', '#refugeeintegration', '#refugeesolidarity', 
    '#refugeeprotection', '#refugeelaw', '#refugeenation', '#worldrefugeeday', '#migration', 
    '#refugeesmatter', '#standupforrefugees', '#refugeeswelcomehere', '#refugeepolicy', '#refugeehealth', 
    '#refugeestories', '#asylum', '#refugeerights', '#refugeelife', '#refugeesarewelcome', '#refugeecrisis', 
    '#refugeesarepeople', '#refugeepolicy', '#refugeestrong', '#withrefugees', '#migrants', 
    '#refugeeshelpingrefugees', '#refugeeawareness', '#migrantes', '#refugeechildren', 
    '#asylumseekers', '#asylum', '#seekingasylum', '#forcedmigration', '#immigration', '#immigrants', '#refugeeprotection', 
    '#refugeestories', '#humanrights', '#refugeecrisis', '#refugeeyouth', '#refugeesolidarity', '#refugeepolicy', 
    '#refugeestrong', '#refugeenation', '#worldrefugeeday', '#refugeeswelcome', '#migration', '#refugeesmatter', 
    '#standupforrefugees', '#refugeeswelcomehere', '#refugeepolicy', '#refugeehealth', '#refugeestories', 
    '#asylum', '#refugeerights', '#refugeelife', '#refugeesarewelcome', '#refugeecrisis', '#refugeesarepeople', 
    '#refugeepolicy', '#refugeestrong', '#withrefugees', '#migrants', '#nonprofit', '#refugeeshelpingrefugees', 
    '#refugeeawareness', '#migrantes', '#refugeechildren', '#asylumseekers']

# Filter the DataFrame to remove rows with missing values in the 'text' column
concatenated_df = df[df['text'].notna()]

# Filter the DataFrame to keep rows with specific hashtags
df_filtered = concatenated_df[concatenated_df['text'].str.contains('|'.join(selected_hashtags))]

print("Length of filtered DataFrame:", len(df_filtered))

Length of filtered DataFrame: 9081


remove URLs

In [126]:
# Define a regular expression pattern to match URLs
url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

# Remove URLs from the 'text' column using regular expressions
df_filtered.loc[:, 'text'] = df_filtered['text'].str.replace(url_pattern, '', regex=True)

def contains_url(text):
    if text is not None and isinstance(text, str):
        return bool(re.search(url_pattern, text))
    return False


C:\Users\anape\AppData\Local\Temp\ipykernel_10596\4108014762.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, 'text'] = df_filtered['text'].str.replace(url_pattern, '', regex=True)


remove specific words from the column 'text'

In [127]:
# Define a list of specific words you want to remove
specific_words_to_remove = ['RT', 'q' , 'vc', 'htt...', 'http...', 'http', 'https', 'htt', 'htt', 'p', 'pq', 'd', 'in', 'tá', 'km', 'n', 'aí', 'ai', 'R', 'h', 'etc', 'la', 'lá', 'at', 'e']

# Create a regular expression pattern to match the specific words
pattern = r'\b(?:' + '|'.join(specific_words_to_remove) + r')\b'

# Remove the specific words from the 'text' column using regular expressions
df_filtered.loc[:, 'text'] = df_filtered['text'].str.replace(pattern, '', regex=True)

C:\Users\anape\AppData\Local\Temp\ipykernel_10596\3675029627.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, 'text'] = df_filtered['text'].str.replace(pattern, '', regex=True)


remove usernames and mentions (@)

In [128]:
# Define a regular expression pattern to match mentions (usernames)
mention_pattern = r'@\w+'

# Remove mentions from the 'text' column using regular expressions
df_filtered.loc[:, 'text'] = df_filtered['text'].str.replace(mention_pattern, '', regex=True)

C:\Users\anape\AppData\Local\Temp\ipykernel_10596\2295321774.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, 'text'] = df_filtered['text'].str.replace(mention_pattern, '', regex=True)


tokenization

In [129]:
# Tokenize the text in the 'text' column using NLTK
df_filtered['tokens'] = df_filtered['text'].apply(word_tokenize)

C:\Users\anape\AppData\Local\Temp\ipykernel_10596\182718224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['tokens'] = df_filtered['text'].apply(word_tokenize)


remove stopwords

In [130]:
additional_stopwords = ['a', 'à', 'agora', 'ainda', 'além', 'algo', 'algumas', 'alguns', 'ali', 'ano', 'anos', 'antes', 'ao', 'aos', 'apenas',
    'apoio', 'após', 'aquela', 'aquelas','aqui', 'aquele', 'aqueles', 'aquilo', 'área', 'as', 'às', 'assim', 'até', 'atrás',
    'através', 'baixo', 'bastante', 'bem', 'boa', 'boas', 'bom', 'bons', 'breve', 'cá', 'cada', 'catorze', 'cedo', 'cento',
    'certamente', 'certeza', 'cinco', 'coisa', 'coisas', 'com', 'como', 'conselho', 'contra', 'contudo', 'corrente',
    'cuja', 'cujas', 'cujo', 'cujos', 'da', 'dá', 'dão', 'daquela', 'daquelas', 'daquele', 'daqueles', 'daqui', 'daquilo',
    'das', 'de', 'debaixo', 'dela', 'delas', 'dele', 'deles', 'dêm', 'demais', 'dentro', 'depois', 'desde', 'dessa',
    'dessas', 'desse', 'desses', 'desta', 'destas', 'deste', 'destes', 'deve', 'deverá', 'dez', 'dezenove', 'dezesseis',
    'dezassete', 'dezoito', 'dia', 'diante', 'diz', 'dizem', 'do', 'dona', 'donas', 'dono', 'donos', 'dos', 'doze', 'duas',
    'dúvida', 'e', 'ela', 'elas', 'ele', 'eles', 'em', 'embora', 'entre', 'então', 'entanto', 'era', 'eram', 'éramos',
    'é', 'essa', 'essas', 'esse', 'esses', 'esta', 'estamos', 'está', 'estão', 'estar', 'estará', 'estas', 'estás', 'estava',
    'estavam', 'este', 'estes', 'esteve', 'estive', 'estivemos', 'estiveram', 'estiveste', 'estivestes', 'estou', 'eu',
    'exemplo', 'falta', 'fará', 'favor', 'faz', 'fazeis', 'fazem', 'fazemos', 'fazer', 'fazes', 'fez', 'fim', 'final', 'foi',
    'fomos', 'for', 'fora', 'foram', 'formos', 'fosse', 'fossem', 'foste', 'fostes', 'fui', 'geral', 'grande', 'grandes',
    'grupo', 'hoje', 'hora', 'horas', 'ir', 'irá', 'isso', 'isto', 'já', 'lá', 'lado', 'ligado', 'local', 'logo', 'longe',
    'lugar', 'maior', 'maioria', 'maiorias', 'mais', 'mal', 'mas', 'máximo', 'me', 'meio', 'menor', 'menos', 'mês',
    'meses', 'meu', 'meus', 'mil', 'minha', 'minhas', 'momento', 'muito', 'muitos', 'na', 'nada', 'naquela', 'naquelas',
    'naquele', 'naqueles', 'nas', 'nem', 'nenhuma', 'nessa', 'nessas', 'nesse', 'nesses', 'nesta', 'nestas', 'neste', 'nestes',
    'ninguém', 'no', 'nos', 'nós', 'nossa', 'nossas', 'nosso', 'nossos', 'nova', 'novas', 'nove', 'novo', 'novos', 'num',
    'numa', 'número', 'nunca', 'nuns', 'o', 'obra', 'obrigada', 'obrigado', 'oitava', 'oitavo', 'oito', 'onde', 'ontem',
    'onze', 'os', 'ou', 'outra', 'outras', 'outro', 'outros', 'para', 'pra','parece', 'parte', 'partir', 'paucas', 'pela', 'pelas',
    'pelo', 'pelos', 'perto', 'pode', 'pôde', 'podem', 'poderá', 'podia', 'pois', 'ponto', 'pontos', 'por', 'porque', 'porquê',
    'pouca', 'pouco', 'poucos', 'primeira', 'primeiras', 'primeiro', 'primeiros', 'própria', 'próprias', 'próprio', 'próprios',
    'quáis', 'qual', 'qualquer', 'quando', 'quanto', 'quarta', 'quarto', 'quatro', 'que', 'quem', 'quer', 'quê', 'quinta',
    'quinto', 'quinze', 'relação', 'sabe', 'são', 'se', 'segunda', 'segundo', 'sei', 'seis', 'seja', 'sejam', 'sempre', 'sendo',
    'ser', 'será', 'seu', 'seus', 'sexta', 'sexto', 'sim', 'sistema', 'sob', 'sobre', 'sois', 'somos', 'sou', 'sua', 'suas',
    'tal', 'talvez', 'também', 'tanta', 'tantas', 'tanto', 'tantos', 'te', 'tem', 'têm', 'temos', 'tendes', 'tenho', 'tens',
    'ter', 'terá', 'terão', 'terceira', 'terceiro', 'teu', 'teus', 'teve', 'ti', 'tido', 'tinha', 'tinham', 'tive', 'tivemos',
    'tiveram', 'tiveste', 'tivestes', 'toda', 'todas', 'todo', 'todos', 'trabalho', 'três', 'treze', 'tu', 'tua', 'tuas',
    'tudo', 'última', 'últimas', 'último', 'últimos', 'um', 'uma', 'umas', 'uns', 'usa', 'usar', 'vai', 'vais', 'valor',
    'veja', 'vem', 'vens', 'ver', 'verdade', 'verdadeiro', 'vez', 'vezes', 'viagem', 'vindo', 'vinte', 'você', 'vocês',
    'vos', 'vós', 'vossa', 'vossas', 'vosso', 'vossos', 'zero', 'via']

# Combine both lists of stopwords
portuguese_stopwords = stop + additional_stopwords

df_filtered['filtered_tokens'] = df_filtered['tokens'].apply(lambda tokens: [word for word in tokens if word.lower() not in portuguese_stopwords])

C:\Users\anape\AppData\Local\Temp\ipykernel_10596\999958453.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['filtered_tokens'] = df_filtered['tokens'].apply(lambda tokens: [word for word in tokens if word.lower() not in portuguese_stopwords])


In [131]:
def join_tokens_into_phrase(tokens):
    return ' '.join(tokens)

# Define a pattern to remove punctuation
punctuation_pattern = r'[^\w\s]'

# Define a list of specific punctuation signs to remove
specific_punctuation = [':', ';', '.', ',']

# Remove the specified punctuation from filtered tokens
def remove_specific_punctuation(tokens):
    return [re.sub(punctuation_pattern, '', word) for word in tokens if word not in specific_punctuation]

df_filtered['cleaned_tokens'] = df_filtered['filtered_tokens'].apply(remove_specific_punctuation)

# Combine cleaned tokens into a list
df_filtered['cleaned_text'] = df_filtered['cleaned_tokens'].apply(join_tokens_into_phrase)

print(df_filtered.head())


                       id                                               text  \
592   1000324601359060994   Estes homens, na grande maioria, não vão ter ...   
625   1000375864557801472  #Portugal. Procura de emprego facilitada aos #...   
713   1000399928542232576  “À solidariedade para com os #migrantes  os #r...   
3874  1002106633722499072   : #Lampedusa: #oração dos bispos africanos pe...   
3983  1002204338146697216  “Encorajo a que se faça tudo o possível para s...   

               created_at           author_id lang in_reply_to_user_id  \
592   2018-05-26 10:35:44          2852272811   pt            29463124   
625   2018-05-26 13:59:26          4878748774   pt                None   
713   2018-05-26 15:35:04  819210050581524480   pt                None   
3874  2018-05-31 08:36:54           368666099   pt                None   
3983  2018-05-31 15:05:08  819210050581524480   pt                None   

                                                 tokens  \
592   [Estes, h

C:\Users\anape\AppData\Local\Temp\ipykernel_10596\2445851292.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['cleaned_tokens'] = df_filtered['filtered_tokens'].apply(remove_specific_punctuation)
C:\Users\anape\AppData\Local\Temp\ipykernel_10596\2445851292.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['cleaned_text'] = df_filtered['cleaned_tokens'].apply(join_tokens_into_phrase)


# 2 - Translation

In [132]:
import translators as ts
from tqdm import tqdm
import time
from googletrans import Translator

DAI'S function - it does not translate and the new created file does not have new columns :( + IF sleep_seconds is less 0.5, google gives time out

In [136]:
# Define function to translate text to English
def translate(x, translator):
    translated_text = ts.translate_text(
        x,
        translator=translator,
        sleep_seconds=0.5,
        timeout=5,
        limit_of_length=40000
    )
    return translated_text

# Assuming df_filtered is your DataFrame and 'cleaned_text' is the column to be translated
data = df_filtered['cleaned_text'].tolist()

# Define the batch size for saving to separate CSV files
batch_size = 1000

# Initialize variables to keep track of progress
current_batch = 1
start_index = 0

while start_index < len(data):
    end_index = start_index + batch_size
    current_data = data[start_index:end_index]

    new_translated_texts = []

    switch = True
    while switch:
        for text, i in zip(current_data, tqdm(range(start_index, end_index), desc=f'Translating {len(current_data)} texts 🐱')):
            translated_text = translate(text, 'google')
            new_translated_texts.append(translated_text)

            if len(new_translated_texts) % 500 == 0:
                switch = False
                print(f'Done translating {len(new_translated_texts)} texts ✨. Have a 100-second break 🐱')
                time.sleep(100)
                switch = True

        if len(new_translated_texts) == len(current_data):
            print('Done translating', len(new_translated_texts), 'texts ✨')
            break

    # Add the new translated column to current_data
    current_df = pd.DataFrame({
        'cleaned_text': current_data,
        'translated_text': new_translated_texts
    })

    # Merge current batch with df_filtered
    merged_df = pd.merge(df_filtered, current_df, left_on='cleaned_text', right_on='cleaned_text')

    # Save the current batch along with all columns to a CSV file
    output_csv_filename = f'translated_df_batch_{current_batch}.csv'
    merged_df.to_csv(output_csv_filename, index=False, encoding='utf-8')

    print(f'Saved batch {current_batch} to {output_csv_filename} ✨')

    # Update progress variables
    current_batch += 1
    start_index = end_index

print('Translation and saving completed ✨')


Translating 1000 texts 🐱:   1%|          | 6/1000 [00:04<13:36,  1.22it/s]


KeyboardInterrupt: 

ME TRYING TO DO ANOTHER CODE (IT DOES NOT WORK, IT CREATES A WEIRD FILE)

In [135]:
import time
import pandas as pd
from translate import Translator

def translate_text(text, target_lang='en'):
    translator = Translator(to_lang=target_lang)
    translation = translator.translate(text)
    return translation

# Additional variables for batch processing
batch_size = 250  # Number of rows to process before sleeping
sleep_duration = 100  # Sleep duration in seconds
save_interval = 500  # Number of rows to translate before saving to a new CSV
separator = ';;'    # Separator for translated texts

# Initialize variables
rows_processed = 0
batch_num = 1

# Translate the 'text' column and create a new column 'translated_text'
df_filtered['translated_text'] = ""

for i, row in df_filtered.iterrows():
    translation = translate_text(row['cleaned_text'])
    df_filtered.at[i, 'translated_text'] = translation
    rows_processed += 1

    # Check if the batch size is reached, then sleep
    if rows_processed % batch_size == 0:
        print(f"Processed {rows_processed} rows. Sleeping for {sleep_duration} seconds.")
        time.sleep(sleep_duration)

    # Check if it's time to save to a new CSV
    if rows_processed % save_interval == 0:
        filename = f'translated_batch_{batch_num}.csv'
        
        # Join translated texts with the separator
        df_filtered['translated_text'] = df_filtered['translated_text'].str.cat(sep=separator)
        
        # Save to CSV
        df_filtered.to_csv(filename, index=False)
        
        print(f"Saved {rows_processed} rows to {filename}.")
        batch_num += 1

    # Add a timer to control the rate of requests
    time.sleep(0.5)

# Save the final batch if needed
if rows_processed % save_interval != 0:
    filename = f'translated_batch_{batch_num}.csv'
    
    # Join translated texts with the separator
    df_filtered['translated_text'] = df_filtered['translated_text'].str.cat(sep=separator)
    
    # Save to CSV
    df_filtered.to_csv(filename, index=False)
    
    print(f"Saved {rows_processed} rows to {filename}.")

print("Translation complete.")


C:\Users\anape\AppData\Local\Temp\ipykernel_10596\1168782258.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['translated_text'] = ""


Processed 250 rows. Sleeping for 100 seconds.
Processed 500 rows. Sleeping for 100 seconds.


C:\Users\anape\AppData\Local\Temp\ipykernel_10596\1168782258.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['translated_text'] = df_filtered['translated_text'].str.cat(sep=separator)


Saved 500 rows to translated_batch_1.csv.
Processed 750 rows. Sleeping for 100 seconds.


KeyboardInterrupt: 